# 0目录
1. SQL基础
2. 合并

# 1 SQL 基础
## 1.1读取csv文件

In [ ]:
data  t_user_test;
    infile 't_user_test.csv' dlm=',' firstobs = 2;
    input uid 
        age 
        sex 
        active_date $10. 
        limit;
run;
 
proc print data=t_user_test;
    title '用户数据表 t_user_test';
run;

## 1.2 SQL查询

In [ ]:
proc sql;
    title 'SQL 查询';
    select uid, age, sex, active_date, limit as money
        from t_user_test
        where active_date > '2016-03-01'
        order by active_date desc; /* 默认升序asce */
quit;

## 1.3 group by

In [ ]:
proc sql;
    title 'group by';
    select age, sum(limit) as money /*Group by 若不与sum()连用，sas会将其变换为order by*/
        from t_user_test
        group by age
        having money > 100 /*Having限定输出的观测*/
        order by money desc;
quit;

# 2 SQL 合并
## 2.1 准备数据

In [ ]:
data  t_user;
    infile 't_user.csv' dlm=',' firstobs = 101 obs = 200;
    input uid 
        age 
        sex 
        active_date $10. 
        limit;
run;
 
proc print data=t_user;
    title '用户数据表 t_user';
run;

In [ ]:
data  t_loan_sum;
    infile 't_loan_sum.csv' dlm=',' firstobs = 2;
    input uid 
        month $
        loan_sum;
run;

proc print data=t_loan_sum;
    title '贷款总金额表 t_loan_sum';
run;

## 2.2  合并
### 横向合并

In [ ]:
proc sql;
title '横向合并 join';
    select u.uid, age, sex, active_date, limit, month, loan_sum 
        from t_user as u
        inner join t_loan_sum as l
        on t_user.uid = t_loan_sum.uid
        order by uid;
quit;

how about merge?
用in来控制非交集行

In [ ]:
proc sort data = t_user;
    by uid;
run;

proc sort data = t_loan_sum;
    by uid;
run;

data merge1;
    merge t_user(in = x) t_loan_sum(in = y);
    by uid;
    if x=1 and y=1; /*用if 非where*/
run;

proc print data = merge1;
    title 'merge';
run;

### 纵向合并

In [1]:
proc sql;
    title '纵向合并 union';
    select * from t_user_test
        union 
        select * from t_user;
        where active_date > '2016-03-01'
            order by active_date desc; /* 默认升序asce */
quit;

how about set
1. 双set，后表覆盖前表
2. 单set，前执行前表，在执行后表，不覆盖

In [ ]:
data set1;
    set t_user_test;
    set t_user;
run;

proc print data = set1;
    title '双set，后表覆盖前表';
run;

In [ ]:
data set1;
    set t_user_test t_user;
run;

proc print data = set1;
    title '单set，前执行前表，在执行后表，不覆盖';
run;